In [162]:
import pandas as pd
import numpy as np

In [163]:
path = "/Users/jhenriqueax/Desktop/LeanConvert/raw_data.xlsx"
raw_data = pd.read_excel(path)
display(raw_data.head())

,User ID,Variant,Action,"Event Count (0 = It did NOT happen, 1 = It did happen)"
0,1000331357699,Variant A,Add to Basket,1
1,1000599036208,Variant A,Add to Basket,1
2,1000671353236,Variant A,Add to Basket,1
3,1000723561185,Variant B,Add to Basket,1
4,1000952317928,Variant B,Add to Basket,1


# Undestanding the data

In [164]:
print("Unique actions:")
print(raw_data["Action"].unique())

Unique actions:
['Add to Basket' 'App Downloads' 'Checkout' 'PDP View' 'Purchase']


In [165]:
line_count = len(raw_data)
print(f"Number of lines: {line_count}")

Number of lines: 332635


In [166]:
event_counts = raw_data["Event Count (0 = It did NOT happen, 1 = It did happen)"].value_counts()
print(event_counts)

Event Count (0 = It did NOT happen, 1 = It did happen)
0    204763
1    127872
Name: count, dtype: int64


In [167]:
total_users = raw_data["User ID"].count()
total_unique_users = raw_data["User ID"].nunique()
print(f"Total users: {total_users}")
print(f"Total unique users: {total_unique_users}")

Total users: 332635
Total unique users: 66527


# Processing the data

In [168]:
# Conta o número de visitantes únicos por variante (A ou B) - base para calcular as taxas de conversão (CR).

unique_visitors = raw_data.groupby("Variant")["User ID"].nunique()

In [ ]:
# Define as ações que compõem o funil de conversão.

actions = ["PDP View", "Add to Basket", "Checkout", "Purchase", "App Downloads"]

In [170]:
# Para cada ação do funil, soma a quantidade de eventos realizados (Event Count = 1), agrupado por variante (A ou B).
# O resultado mostra quantas pessoas realizaram cada ação por grupo (A ou B).

event_counts = {}
for action in actions:
    action_data = raw_data[raw_data["Action"] == action]
    
    grouped = action_data.groupby("Variant")["Event Count (0 = It did NOT happen, 1 = It did happen)"].sum()

    event_counts[action] = grouped

In [171]:
# Cria um DataFrame consolidado com as contagens de ações e o número total de visitantes únicos por variante (base para calcular as taxas de conversão).

results_df = pd.DataFrame(event_counts)
results_df["Unique Visitors"] = unique_visitors

In [172]:
print(results_df)

           PDP View  Add to Basket  Checkout  Purchase  App Downloads  \
Variant                                                                 
Variant A     29025          21987      7049      2055           1879   
Variant B     28983          24021      9087      2072           1714   

           Unique Visitors  
Variant                     
Variant A            33374  
Variant B            33153  


In [173]:
# Para cada ação do funil, calcula a taxa de conversão (%).

for action in actions:
    column_name = f"{action} CR (%)"
    
    results_df[column_name] = (results_df[action] / results_df["Unique Visitors"]) * 100

In [174]:
# Cria uma lista alternando entre as colunas de contagem e de taxa de conversão para cada ação
metrics_columns = []

for action in actions:
    metrics_columns.append(action)             # valor absoluto 
    metrics_columns.append(f"{action} CR (%)") # taxa de conversão 
    

ordered_columns = ["Unique Visitors"] + metrics_columns

results_df = results_df[ordered_columns]


In [175]:
pd.set_option('display.float_format', '{:,.2f}'.format)
print(results_df.reset_index().to_string(index=False))

  Variant  Unique Visitors  PDP View  PDP View CR (%)  Add to Basket  Add to Basket CR (%)  Checkout  Checkout CR (%)  Purchase  Purchase CR (%)  App Downloads  App Downloads CR (%)
Variant A            33374     29025            86.97          21987                 65.88      7049            21.12      2055             6.16           1879                  5.63
Variant B            33153     28983            87.42          24021                 72.45      9087            27.41      2072             6.25           1714                  5.17
